In [1]:
# General imports
import pandas as pd

# Grid search
from library.genetic_algorithms.grid_search_parallelized import grid_search_par
from library.genetic_algorithms.mutation import swap_mutation, inversion_mutation, heuristic_mutation, misfit_mutation
from library.genetic_algorithms.crossover import cycle_crossover, one_point_crossover, uniform_crossover, geometric_crossover
from library.genetic_algorithms.selection import fitness_proportionate_selection, ranking_selection, tournament_selection

In [2]:
seating_data= pd.read_csv("seating_data.csv", index_col=0)

In [3]:
relationships_mtx=seating_data.to_numpy()

In [4]:
relationships_mtx

array([[   0, 5000,    0, ...,    0,    0,    0],
       [5000,    0,  700, ...,    0,    0,    0],
       [   0,  700,    0, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,    0,  700,  700],
       [   0,    0,    0, ...,  700,    0,  900],
       [   0,    0,    0, ...,  700,  900,    0]], shape=(64, 64))

# Experiment with different mutation, crossover and selection methods

## Fixed hyperparameters:
- population size = 100
- max generations = 100
- crossover probability = 0.9
- mutation probability = 0.1
- when selection method is tournment: tournment size = 3

In [ ]:
# Testing grid search code 

results = grid_search_par(
    relations_mtx=relationships_mtx,
    mutation_functions=[swap_mutation, inversion_mutation, heuristic_mutation, misfit_mutation],
    crossover_functions=[cycle_crossover, one_point_crossover, uniform_crossover, geometric_crossover],
    selection_functions=[fitness_proportionate_selection, ranking_selection, tournament_selection],
)

results #results is empty, therefore nothing is printed

Each combination (w/ 30 runs) takes around 6 min.
- 48 combinations: 4.8 h.

In [ ]:
results_table1 = pd.DataFrame(results).style \
    .format(precision=3) \
    .set_properties(**{"text-align": "center"}) \
    .set_table_styles([{
        "selector": "th",
        "props": [("text-align", "center")]
    }])

results_table1

[]


# Experiment with different nr of elits for best mutation, crossover and selection method

In [8]:
results = grid_search_par(
    relations_mtx=relationships_mtx,
    mutation_functions=[], # add best mutation
    crossover_functions=[], # add best crossover
    selection_functions=[], # add best selection
    elitism=[0, 5]
)

Each combination (w/ 30 runs) takes around 6 min.
- 2 combinations: 12 min.

In [20]:
results_table2 = pd.DataFrame(results).style \
    .format(precision=3) \
    .set_properties(**{"text-align": "center"}) \
    .set_table_styles([{
        "selector": "th",
        "props": [("text-align", "center")]
    }])

results_table2

# Plot average fitness per gen 

# Performs statistical tests

# Calculate success rate